In [104]:
import torch
from torchvision.transforms import transforms
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import tensorflow as tf

from taskdataset import TaskDataset
from dataset_augmentation import augment_dataset, augment_dataset_other

2024-03-16 20:12:10.826965: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-16 20:12:13.704385: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [108]:

def parse_vector(vector_str):
    return [float(x.strip()) for x in vector_str.strip('[]').split(',')]

df = pd.read_csv('output.csv', sep=";")

df['model_output'] = df['model_output'].apply(parse_vector)
ids = df.iloc[:,0].values
labels = df.iloc[:,1].values
dataset = torch.load("data/ModelStealingPub.pt")
dataset_pt = TaskDataset()
dataset_pt.ids = ids
dataset_pt.imgs = [dataset.imgs[x] for x in ids]
dataset_pt.labels = labels

# print(dataset_pt.labels)

# Rozszerz model

# Uzyskaj dostęp do obrazów i etykiet z wczytanego zestawu danych
imgs = dataset_pt.imgs
labels = dataset_pt.labels
images_rgb = []
for img in imgs:
    if type(img) is not int:
        images_rgb.append(img.convert("RGB"))

# labels_int = [i for i in range(len(dataset.))]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])




# Konwertuj listy obrazów i etykiet na tensory
imgs_tensor = torch.stack([transform(img) for img in images_rgb])

# Convert and stack transformed labels
labels_tensor = [torch.tensor(label) for label in labels]
print(len([labels[0]]))
# Twórz DataLoader bezpośrednio z obrazami i etykietami
train_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(imgs_tensor, labels_tensor),
    batch_size=32, shuffle=True)

# Zdefiniuj model
model = resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(labels))

# Wybierz urządzenie
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Definiuj funkcję straty i optymalizator
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# # Trenuj model
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        print(labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")
torch.save(model.state_dict(), 'wytrenowany_model.pt')

1


AttributeError: 'list' object has no attribute 'size'